In [62]:
import pandas as pd

In [63]:
from MLForecastPipeline import *

def split_data(df, scenario, date_col="ds"):
    """Extracts train and test data based on train end date."""
    train_data = df[df[date_col] <= scenario['train_end']]
    test_start = pd.to_datetime(scenario['train_end']) + pd.Timedelta(days=1)
    test_data = df[df[date_col] >= test_start]
    return train_data, test_data

selected_sensors_df = pd.read_csv("../data/selected_sensors2_cleaned.csv", index_col=0)
testing_df = format_df_to_mlforecast(selected_sensors_df, 'full_date', '2')[['ds', 'y', 'unique_id']]
testing_df['ds'] = pd.to_datetime(testing_df['ds'])
train_df, test_df = split_data(testing_df, {"train_start": "2017-04-01", "train_end": "2018-04-01"})

In [64]:

# from prophet import Prophet
# from sklearn.base import BaseEstimator, RegressorMixin

# class ProphetWrapper(BaseEstimator, RegressorMixin):
#     def __init__(self, **prophet_params):
#         self.prophet_params = prophet_params
#         self.model = None

#     def fit(self, X, y):
#         # Prophet requires a DataFrame with 'ds' and 'y' columns
#         print(X)
#         df = X.copy()
#         df['y'] = y
#         self.model = Prophet(**self.prophet_params)
#         self.model.fit(df)
#         return self

#     def predict(self, X):
#         # Prophet requires a DataFrame with 'ds' column for predictions
#         print("HEYYYYYYYY - \n", X)
#         future = X[['ds']].copy()
#         forecast = self.model.predict(future)

#         # future = self.model.make_future_dataframe(periods=len(X), freq='D')
#         # forecast = self.model.predict(future)

#         return forecast['yhat'].values

#     def get_params(self, deep=True):
#         return self.prophet_params

#     def set_params(self, **parameters):
#         for parameter, value in parameters.items():
#             setattr(self, parameter, value)
#         return self
    
# models = {
#     "Prophet": ProphetWrapper()
# }
# model_name = 'Prophet'

# fcst = MLForecast(
#     models=[models[model_name]],
#     freq='D',
#     lags=[0],
#     # target_transforms=list(transform_combination),
#     # date_features=date_features,
#     num_threads=1,
#     # lag_transforms=lag_transforms,
# )

# # Fit the model
# fcst.fit(train_df)

# # Predict
# predictions = fcst.predict(h=test_df.shape[0])
# test_df_copy = test_df.copy()
# test_df_copy['forecast'] = predictions[model_name].values     

In [65]:
scenarios_sensors = {
    # 0: 1, 4372603
    # "0_12M_train_7M_test": {"train_start": "2017-03-25", "train_end": "2018-03-25", "test_start": "2018-03-26", "test_end": "2018-10-10"},
    '2': {
        "26M_train":  {"train_start": "2017-04-01", "train_end": "2019-06-01"},
        "24M_train":  {"train_start": "2017-04-01", "train_end": "2019-04-01"},
        "18M_train":  {"train_start": "2017-04-01", "train_end": "2018-10-01"},
        "12M_train":  {"train_start": "2017-04-01", "train_end": "2018-04-01"},
        "8M_train":   {"train_start": "2017-04-01", "train_end": "2017-10-25"},
        
        # Non-Heating Periods
        "NH_3M_train":  {"train_start": "2017-04-15", "train_end": "2017-07-15"},
        "NH_4M_train":  {"train_start": "2017-04-15", "train_end": "2017-08-15"},
        },
}

In [66]:
from itertools import combinations, chain
import pandas as pd
from prophet import Prophet
from copy import deepcopy

def evaluate_models_prophet(train_df, test_df, target_transforms):
    """
    Evaluates multiple models with different transformations using the AutoSeasonalityAndDifferences API.
    Applies transformations to test data and inverse transforms predictions for evaluation.
    """
    results = []

    # Validate transform combinations
    valid_transform_combinations = [()] + list(chain(combinations(target_transforms, 1), combinations(target_transforms, 2)))
    valid_transform_combinations = [tc for tc in valid_transform_combinations if filter_conflicting_transforms(tc)]

    max_test_length = len(test_df)  # Full test period

    # Define test segment lengths: 1-6 months, then 8, 10, 12, 16, 20, etc.
    test_lengths = list(range(30, 181, 30)) + [240, 300, 360, 480, 600, 720, max_test_length]  # Days-based segmentation

    # Filter lengths within available test period
    test_lengths = [t for t in test_lengths if t <= max_test_length]

    total_fits = len(valid_transform_combinations)
    print(f"Total model fits to run: {total_fits}")

    fit_num = 0

    train_df = train_df[['ds', 'y']].copy()
    test_df = test_df[['ds', 'y']].copy()

    for transform_combination in valid_transform_combinations:
        print(f"{fit_num}/{total_fits} Training...")

        # try:
        # Apply transformations to training data
        transformed_train_df = train_df.copy()
        transformers = []  # Store transformers for inverse transformation later
        for transform in transform_combination:
            # Initialize the transformer
            transformer = deepcopy(transform)
            # Fit and transform the training data
            transformed_train_df['y'] = transformer.fit_transform(transformed_train_df['y'])
            transformers.append(transformer)  # Save the transformer

        # Initialize and fit the model
        model = Prophet(seasonality_mode='multiplicative', yearly_seasonality=True, weekly_seasonality=True)
        model.fit(transformed_train_df)

        # Transform test data using the same transformers
        transformed_test_df = test_df.copy()
        for transformer in transformers:
            transformed_test_df['y'] = transformer.transform(transformed_test_df['y'])

        # Predict on transformed test data
        future = model.make_future_dataframe(periods=max_test_length)
        predictions = model.predict(future)
        transformed_test_df['forecast'] = predictions['yhat'].values[-max_test_length:]

        # Inverse transform predictions to bring them back to the original scale
        inverse_transformed_predictions = transformed_test_df['forecast'].copy()
        for transformer in reversed(transformers):  # Apply inverse in reverse order
            inverse_transformed_predictions = transformer.inverse_transform(inverse_transformed_predictions)

        # Add inverse-transformed predictions to the test dataframe
        test_df_copy = test_df.copy()
        test_df_copy['forecast'] = inverse_transformed_predictions

        error_dict = {}

        for test_length in test_lengths:
            eval_subset = test_df_copy.iloc[:test_length]  # Take subset for evaluation
            error_dict[f"test_{test_length}_days"] = mape_met(eval_subset['y'].values, eval_subset['forecast'].values)

        # Store results
        results.append({
            "Model": "Prophet",
            "Transforms": stringify_transform(list(transform_combination)),
            **error_dict  # Expand error dictionary into separate columns
        })
        print(f"Prophet MAPE: {error_dict[f'test_{max_test_length}_days']:.2f}% with transforms {transform_combination}")
            
        # except Exception as e:
            # print(f"Skipping combination {fit_num} due to error: {e}")

        fit_num += 1

    return pd.DataFrame(results)

In [ ]:
from utilsforecast.processing import counts_by_id
import pandas as pd
from itertools import chain, combinations
from prophet import Prophet
from mlforecast.target_transforms import Differences, LocalStandardScaler
from utilsforecast.processing import counts_by_id
from mlforecast.utils import generate_daily_series
from coreforecast.grouped_array import GroupedArray

def dataframe_to_grouped_array(df, id_col, target_col):
    """
    Converts a pandas DataFrame to a GroupedArray required by mlforecast transformations.
    """
    id_counts = counts_by_id(df, id_col)
    indptr = np.append(0, id_counts['counts'].cumsum())
    return GroupedArray(df[target_col].values, indptr)

def apply_transformations(ga, transformations):
    """
    Applies a series of transformations to a GroupedArray.
    """
    for transform in transformations:
        ga = transform.fit_transform(ga)
    return ga

def inverse_transformations(ga, transformations):
    """
    Applies inverse transformations to a GroupedArray in reverse order.
    """
    for transform in reversed(transformations):
        ga = transform.inverse_transform(ga)
    return ga

In [ ]:
from itertools import combinations, chain
import pandas as pd
from prophet import Prophet
from utilsforecast.processing import counts_by_id
import pandas as pd
from itertools import chain, combinations
from prophet import Prophet
from mlforecast.target_transforms import Differences, LocalStandardScaler
from utilsforecast.processing import counts_by_id
from mlforecast.utils import generate_daily_series
from coreforecast.grouped_array import GroupedArray

def dataframe_to_grouped_array(df, id_col, target_col):
    """
    Converts a pandas DataFrame to a GroupedArray required by mlforecast transformations.
    """
    id_counts = counts_by_id(df, id_col)
    indptr = np.append(0, id_counts['counts'].cumsum())
    return GroupedArray(df[target_col].values, indptr)

def apply_transformations(ga, transformations):
    """
    Applies a series of transformations to a GroupedArray.
    """
    for transform in transformations:
        ga = transform.fit_transform(ga)
    return ga

def inverse_transformations(ga, transformations):
    """
    Applies inverse transformations to a GroupedArray in reverse order.
    """
    for transform in reversed(transformations):
        ga = transform.inverse_transform(ga)
    return ga

# Ensure kazakhstan_holidays is a DataFrame with 'ds' and 'holiday' columns
kazakhstan_holidays = pd.read_csv("../data/kazakhstan_holidays.csv")

def evaluate_models_prophet(train_df, test_df, target_transforms):
    """
    Evaluates multiple Prophet models with different configurations:
    - Base model without additional seasonality or holidays.
    - Model with additional seasonality.
    - Model with holidays.
    - Model with both additional seasonality and holidays.
    Applies transformations to test data and inverse transforms predictions for evaluation.
    """
    results = []

    # Define model configurations
    model_configs = [
        {"name": "prophet", "seasonality": False, "holidays": False},
        {"name": "prophet_add_season30_5", "seasonality": True, "holidays": False},
        {"name": "prophet_holy", "seasonality": False, "holidays": True},
        {"name": "prophet_add_season30_5_holy", "seasonality": True, "holidays": True}
    ]

    # Validate transform combinations
    valid_transform_combinations = [()] + list(chain(combinations(target_transforms, 1), combinations(target_transforms, 2)))
    valid_transform_combinations = [tc for tc in valid_transform_combinations if filter_conflicting_transforms(tc)]

    max_test_length = len(test_df)  # Full test period

    # Define test segment lengths: 1-6 months, then 8, 10, 12, 16, 20, etc.
    test_lengths = list(range(30, 181, 30)) + [240, 300, 360, 480, 600, 720, max_test_length]  # Days-based segmentation

    # Filter lengths within available test period
    test_lengths = [t for t in test_lengths if t <= max_test_length]

    total_fits = len(valid_transform_combinations) * len(model_configs)
    print(f"Total model fits to run: {total_fits}")

    fit_num = 0

    for config in model_configs:
        for transform_combination in valid_transform_combinations:
            print(f"{fit_num + 1}/{total_fits} Training {config['name']} with transforms: {stringify_transform(transform_combination)}...")

            try:
                # Convert training data to GroupedArray
                train_ga = dataframe_to_grouped_array(train_df, 'unique_id', 'y')

                # Apply transformations
                transformed_train_ga = apply_transformations(train_ga, transform_combination)

                # Prepare transformed training DataFrame
                transformed_train_df = train_df.copy()
                transformed_train_df['y'] = transformed_train_ga.data

                # Initialize the Prophet model
                model = Prophet(
                    seasonality_mode='multiplicative',
                    yearly_seasonality=True,
                    weekly_seasonality=True,
                    holidays=kazakhstan_holidays if config['holidays'] else None
                )

                # Add additional seasonality if specified
                if config['seasonality']:
                    model.add_seasonality(name='monthly', period=30.5, fourier_order=8)

                # Fit the model on transformed training data
                model.fit(transformed_train_df[['ds', 'y']])

                # Prepare a DataFrame for future dates
                future = model.make_future_dataframe(periods=max_test_length, freq='D')
                forecast = model.predict(future)

                # Extract the forecasted values corresponding to the test period
                transformed_forecast = forecast[['ds', 'yhat']].iloc[-max_test_length:].copy()
                transformed_forecast.rename(columns={'yhat': 'y'}, inplace=True)
                transformed_forecast['unique_id'] = test_df['unique_id'].iloc[0]  # Assuming a single series

                # Convert forecast to GroupedArray
                forecast_ga = dataframe_to_grouped_array(transformed_forecast, 'unique_id', 'y')

                # Inverse transform the forecasted values
                inverse_transformed_forecast_ga = inverse_transformations(forecast_ga, transform_combination)

                # Align the inverse-transformed forecasts with the test data
                test_df_copy = test_df.copy()
                test_df_copy['forecast'] = inverse_transformed_forecast_ga.data

                error_dict = {}

                for test_length in test_lengths:
                    eval_subset = test_df_copy.iloc[:test_length]  # Take subset for evaluation
                    error_dict[f"test_{test_length}_days"] = mape_met(eval_subset['y'].values, eval_subset['forecast'].values)

                # Store results
                results.append({
                    "Model": config['name'],
                    "Transforms": stringify_transform(list(transform_combination)),
                    **error_dict  # Expand error dictionary into separate columns
                })
                print(f"{config['name']} MAPE: {error_dict[f'test_{max_test_length}_days']:.2f}% with transforms {stringify_transform(list(transform_combination))}")
            except Exception as e:
                print(f"Skipping combination {fit_num + 1} due to error: {e}")
            fit_num += 1

    return pd.DataFrame(results)


In [74]:
# Loop through scenarios and evaluate models
results = []

for sensor_name, scenarios in scenarios_sensors.items():
    formatted_df = format_df_to_mlforecast(selected_sensors_df[['full_date', sensor_name]], 'full_date', sensor_name, unique_id=sensor_name)
    formatted_df = formatted_df[['ds', 'y', 'unique_id']]

    for scenario_name, scenario in scenarios.items():

        train_df, test_df = split_data(formatted_df, scenario)

        target_transforms = get_dynamic_transforms(train_df)
        results = evaluate_models_prophet(train_df, test_df, target_transforms)

        save_results(results, f"results/run_4/{sensor_name}_{scenario_name}.csv") 

12:15:01 - cmdstanpy - INFO - Chain [1] start processing
12:15:01 - cmdstanpy - INFO - Chain [1] done processing


Total model fits to run: 64
1/64 Training prophet with transforms: tuple()...


12:15:02 - cmdstanpy - INFO - Chain [1] start processing
12:15:02 - cmdstanpy - INFO - Chain [1] done processing


prophet MAPE: 41.33% with transforms 
2/64 Training prophet with transforms: tuple()...


12:15:02 - cmdstanpy - INFO - Chain [1] start processing
12:15:02 - cmdstanpy - INFO - Chain [1] done processing


...

12:19:52 - cmdstanpy - INFO - Chain [1] start processing
12:19:52 - cmdstanpy - INFO - Chain [1] done processing


prophet_holy MAPE: 394.98% with transforms AutoSeasonalDifferences(max_diffs=279, n_seasons=10, season_length=186) | LocalStandardScaler(stats_=[[38.77238083 33.27355631]])
44/64 Training prophet_holy with transforms: tuple()...


12:19:52 - cmdstanpy - INFO - Chain [1] start processing
12:19:53 - cmdstanpy - INFO - Chain [1] done processing


prophet_holy MAPE: 39.54% with transforms AutoSeasonalDifferences(max_diffs=279, n_seasons=10, season_length=186) | LocalMinMaxScaler(stats_=[[  5.11211699 183.47202765]])
45/64 Training prophet_holy with transforms: tuple()...
prophet_holy MAPE: 73.86% with transforms AutoSeasonalDifferences(max_diffs=279, n_seasons=10, season_length=186) | LocalBoxCox(lower=0.0, method=loglik, season_length=None, stats_=[[4.10226185e-005 2.76182696e-321]], upper=2.0)
46/64 Training prophet_holy with transforms: tuple()...


KeyboardInterrupt: 

## waqi data

In [8]:
file_path = r"C:\Users\77019\Downloads\waqi-covid19-airqualitydata-2025.csv"

# Read CSV while skipping comment lines
data = pd.read_csv(file_path, comment='#')
data = data.loc[data['City'] == "Almaty"]

In [10]:
data['Specie'].unique()

array(['dew', 'co', 'humidity', 'so2', 'pressure', 'wind-speed',
       'wind-gust', 'temperature', 'pm10', 'pm25', 'no2'], dtype=object)

In [13]:
data['Date'].min(), data['Date'].max()

('2024-01-01', '2025-03-05')

In [14]:
import requests
import time

# List of available periods
periods = [
    "2015H1", "2016H1", "2017H1", "2018H1",
    "2019Q1", "2019Q2", "2019Q3", "2019Q4",
    "2020Q1", "2020Q2", "2020Q3", "2020Q4",
    "2021Q1", "2021Q2", "2021Q3", "2021Q4",
    "2022Q1", "2022Q2", "2022Q3", "2022Q4",
    "2023Q1", "2023Q2", "2023Q3", "2023Q4"
]

# Base URL
base_url = "https://aqicn.org/data-platform/covid19/report/45268-77d0de2d"

# Loop through each period and download the file
for period in periods:
    url = f"{base_url}/{period}"
    file_name = f"waqi-airquality-{period}.csv"

    print(f"Downloading {period}...")

    response = requests.get(url)

    if response.status_code == 200:
        with open(file_name, "wb") as file:
            file.write(response.content)
        print(f"✅ {file_name} downloaded successfully!")
    else:
        print(f"❌ Failed to download {period}. Status code: {response.status_code}")

    # Add a small delay to prevent overwhelming the server
    time.sleep(2)

print("🎉 All available datasets have been processed.")


✅ waqi-airquality-2015H1.csv downloaded successfully!
✅ waqi-airquality-2016H1.csv downloaded successfully!
✅ waqi-airquality-2017H1.csv downloaded successfully!
✅ waqi-airquality-2018H1.csv downloaded successfully!
✅ waqi-airquality-2019Q1.csv downloaded successfully!
✅ waqi-airquality-2019Q2.csv downloaded successfully!
✅ waqi-airquality-2019Q3.csv downloaded successfully!
✅ waqi-airquality-2019Q4.csv downloaded successfully!
✅ waqi-airquality-2020Q1.csv downloaded successfully!
✅ waqi-airquality-2020Q2.csv downloaded successfully!
✅ waqi-airquality-2020Q3.csv downloaded successfully!
✅ waqi-airquality-2020Q4.csv downloaded successfully!
✅ waqi-airquality-2021Q1.csv downloaded successfully!
✅ waqi-airquality-2021Q2.csv downloaded successfully!
✅ waqi-airquality-2021Q3.csv downloaded successfully!
✅ waqi-airquality-2021Q4.csv downloaded successfully!
✅ waqi-airquality-2022Q1.csv downloaded successfully!
✅ waqi-airquality-2022Q2.csv downloaded successfully!
✅ waqi-airquality-2022Q3.csv